## Création de la base de données de recettes ##
Ce fichier python permet de préparer mes données. J'ai utilisé le package Python recipe-scrapers qui permet d'extraire les données d'une recette provenant d'un site internet de cuisine parmi une sélection de sites. 
J'ai choisit d'extraire les recettes de muffins du site Marmiton, car il contient un grand nombre de recette **en français**.

In [2]:
# Première fonction test sur une recette de muffin du site Marmiton

from recipe_scrapers import scrape_me

scraper = scrape_me("https://www.marmiton.org/recettes/recette_muffins-a-la-banane_14745.aspx")
scraper.title()
scraper.instructions()
scraper.to_json()


{'author': 'BIGSTAR MEMEL',
 'canonical_url': 'https://www.marmiton.org/recettes/recette_muffins-a-la-banane_14745.aspx',
 'category': 'muffin',
 'cook_time': 20,
 'cuisine': 'Dessert',
 'description': 'banane, sucre, oeuf, bicarbonate, levure, sel, farine, beurre',
 'host': 'marmiton.org',
 'image': 'https://assets.afcdn.com/recipe/20150225/373_w1024h576c1cx1250cy1745.jpg',
 'ingredient_groups': [{'ingredients': ['3 bananes',
    '75 g de sucre blanc',
    '1 oeufs',
    '1 cuillères à café de bicarbonate',
    '1 cuillères à café de levure',
    '1/2 cuillères à café de sel',
    '150 g de farine',
    '70 g de beurre fondu'],
   'purpose': None}],
 'ingredients': ['3 bananes',
  '75 g de sucre blanc',
  '1 oeufs',
  '1 cuillères à café de bicarbonate',
  '1 cuillères à café de levure',
  '1/2 cuillères à café de sel',
  '150 g de farine',
  '70 g de beurre fondu'],
 'instructions': "Ecraser les bananes. Ajouter le sucre et l'oeuf légèrement battu. Ajouter le beurre fondu, puis les i

In [1]:
# Généralisation sur toutes les recettes de muffins

import requests
from bs4 import BeautifulSoup
from recipe_scrapers import scrape_me
import time

def get_muffins(recherche="muffin"):
    base_url = f"https://www.marmiton.org/recettes/recherche.aspx?aqt={recherche}" # On filtre la recherche, l'url doit contenir le mot 'muffin'
    base_de_donnees = [] # pour stocker la base de données
    page = 1
    continuer = True
    headers = {'User-Agent': 'Mozilla/5.0'}


    while continuer:
        print(f"Analyse de la page {page}")
        url_page = f"{base_url}&page={page}"
        response = requests.get(url_page, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')  # transforme le texte HTML proprement
        
        # On cherche les liens de recettes
        liens = [a.get('href') for a in soup.select('a[href*="/recettes/recette_"]')]
        
        # si pas de lien trouvé : fin des résultats
        if not liens:
            print("Fin des résultats atteinte.")
            continuer = False
            break

        nombre_recettes = 0
        for href in set(liens):
            full_url = "https://www.marmiton.org" + href if href.startswith('/') else href
            
            try:
                nombre_recettes += 1
                scraper = scrape_me(full_url)
                titre = scraper.title()
                
                # On garde le filtre "muffin" dans le titre
                if "muffin" in titre.lower():
                    recette = {
                        "titre": scraper.title(),
                        "categorie": scraper.category(),
                        "temps": scraper.total_time(),
                        "description": scraper.description(),
                        "ingredients": scraper.ingredients(),
                        "instructions": scraper.instructions(),
                        "langue": scraper.language(),
                        "site": scraper.site_name()
                    }
                    base_de_donnees.append(recette)
                    print(f"✅ {titre}")
                
                time.sleep(0.4) 

            except Exception as e:
                # Si une recette plante, on passe à la suivante sans arrêter tout le script
                continue

        # Passage à la page suivante
        page += 1
        
        # On s'arrête à 25 pages max 
        if page > 25: 
            continuer = False

    return base_de_donnees, nombre_recettes

# Lancement
base_de_donnees, nombre_recettes = get_muffins("muffin")

Analyse de la page 1
✅ Muffins très simples
✅ Muffins poires pépites de chocolat de Nadine
✅ Muffins à la banane
✅ Muffins framboises et chocolat blanc
✅ Muffins coeur de Nutella
✅ Muffins aux pépites de chocolat
✅ Muffins aux pommes
✅ Muffins pomme cannelle gingembre
✅ Muffins de Sandrine
✅ Muffins surprises au chocolat
✅ Muffins à la pâte à tartiner et aux pépites de chocolat
✅ Muffins allégés aux pépites de chocolat
✅ Muffins aux framboises
✅ Muffins au jambon et olives vertes de Nadine
✅ Muffins à la framboise de Jeanne
Analyse de la page 2
✅ Mes muffins banane chocolat blanc
✅ Muffins tout chocolat
✅ Muffins au miel
✅ Muffins aux pépites de chocolat, noix et cannelle
✅ Muffin bananes et pépites de chocolat
✅ Muffins à la banane faciles
✅ muffins choco-banane
✅ Muffins aux myrtilles
✅ Muffins aux fraises et à la menthe
✅ Muffins pommes canelle
✅ Mini muffins choco-noisettes
✅ Muffin chocolat noir au coeur fondant de chocolat blanc
✅ Muffins salés poulet-curry
✅ Mini-muffins fondant

In [2]:
# Créer un fichier json contenant la base de données

import json

with open('base_de_donnees.json', 'w', encoding='utf-8') as f:
    json.dump(base_de_donnees, f, indent=4, ensure_ascii=False)

print("Fichier base_de_donnees.json créé !")

Fichier base_de_donnees.json créé !


In [6]:
# Afficher le nombre de recettes dans ma base de données 

print(f"La base de données contient {len(base_de_donnees)} recettes de muffins.")

La base de données contient 375 recettes de muffins.


In [ ]:
# Créer une nouvelle catégorie dans ma base de données appelée text_for_embedding qui contient le titre et la liste des ingrédients
for recette in base_de_donnees:
    titre_pur = recette['titre'].lower().replace('muffins', '').replace('muffin', '').strip()



    ingredients_pur = recette['description'].lower().replace('farine', '').replace('oeufs', '').replace('oeuf', '').replace('sucre', '').replace('lait', '').replace('muffins,', '').replace('muffins', '').replace('muffin,', '').replace('muffin', '').strip() # liste des ingrédients sans muffins
    recette["text_for_embedding"] = f"{titre_pur} {titre_pur} {ingredients_pur}"

    with open("base_de_donnees.json", 'w', encoding='utf-8') as f:
        json.dump(base_de_donnees, f, ensure_ascii=False, indent=4)



In [46]:
# Créer une nouvelle catégorie dans ma base de données appelée text_for_embedding qui contient le titre et la liste des ingrédients
for recette in base_de_donnees:    
    recette["text_for_embedding"] = f"{recette['titre']} {recette['description']}"

    with open("base_de_donnees.json", 'w', encoding='utf-8') as f:
        json.dump(base_de_donnees, f, ensure_ascii=False, indent=4)



In [44]:
# Liste des "mots parasites" qui noient les saveurs (à adapter selon tes besoins)
STOP_WORDS_CUISINE = ["farine", "sucre", "oeuf", "oeufs", "lait", "beurre", "levure", "sel", "huile", "muffin", "muffins", "chimique", "poudre", "d'olive", "crème", "liquide"]

def extraire_adn(titre, description):
    # On nettoie le titre et la description
    texte = f"{description}".lower()
    mots = texte.replace(',', ' ').replace('.', ' ').split()
    
    # On ne garde que les mots qui ont du sens (saveurs, légumes, fruits)
    mots_filtres = [m for m in mots if m not in STOP_WORDS_CUISINE and len(m) > 2]
    
    # On répète le titre au début pour donner le thème
    titre_nettoyé = titre.lower().replace('muffins', '').replace('muffin', '').strip()
    
    return f"{titre_nettoyé} {' '.join(mots_filtres)}"

for recette in base_de_donnees:
    recette["text_for_embedding"] = extraire_adn(recette['titre'], recette['description'])

    with open("base_de_donnees.json", 'w', encoding='utf-8') as f:
        json.dump(base_de_donnees, f, ensure_ascii=False, indent=4)

La base de données a été créée et enregistrer dans le fichier base_de_donnees.json. Elle contient 375 recettes de muffins en français. Les données importantes (titre de la recette, et ingrédients) ont été enregistrées dans "text_for_embedding". 